# Machine learning su aplicación en Biología

__Análisis de datos biológicos utilizando métodos de machine learning__

_MeIA_

`2023`

## Árbol de decisión y Bosques aleatorios

<img src="../imagenes/DECISION-TREE.png" width="400" height="400"/>

Los árboles de decisión y los bosques aleatorios son algoritmos de aprendizaje supervisado que se utilizan para problemas de clasificación y regresión. 

Un árbol de decisión se basa en todo el conjunto de datos, mientras que un bosque aleatorio selecciona de forma aleatoria observaciones y caractéristicas para crear varios árboles de decisión y luego promedia los resultados.

Cada nodo del árbol es una prueba o condición y las ramas representan su resultado.
- Nodo Raíz: Sirve como la población total o la muestra que se divide en dos o más conjuntos diversos de grupos.
- Nodo de decisión: El nodo de decisión se nombra cuando un subnodo se divide en otros subnodos.
- Nodo hoja: Son los resultados finales de todas las decisiones encadenadas respectivas y, a menudo, se denominan nodos terminales.



Primero intentaremos aplicar árboles de decisión y el bosque aleatorio en el conjunto de datos de enfermedades cardíacas.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

In [3]:
dataset = pd.read_csv('../Datos/heart.csv')
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
#Separamos la variables dependiente e independiente
X = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

### Árboles de decisión

Vamos a entrenar el modelo usando la función fit()

In [5]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)

DecisionTreeClassifier()

#### Predicción y evaluación

In [6]:
#Vamos a evaluar nuestro árbol
predictions = classifier.predict(X_test)

In [7]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.74      0.82      0.78        39
           1       0.85      0.79      0.82        52

    accuracy                           0.80        91
   macro avg       0.80      0.80      0.80        91
weighted avg       0.81      0.80      0.80        91



El árbol de decisión alzanzó una tasa de exactitud del 77% que es más alta que la exactitud de referencia (54%).

In [8]:
#### Vamos a predecir un nuevo valor
dataset.iloc[2,:]

age          41.0
sex           0.0
cp            1.0
trestbps    130.0
chol        204.0
fbs           0.0
restecg       0.0
thalach     172.0
exang         0.0
oldpeak       1.4
slope         2.0
ca            0.0
thal          2.0
target        1.0
Name: 2, dtype: float64

In [9]:
new_data=dataset.iloc[2,:-1].values
classifier.predict([new_data])

/Users/galanve/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1])

Conclusión: El modelo del árbol predice correctamente nuestro nuevo valor

### Random Forests

In [10]:
from sklearn.ensemble import RandomForestClassifier
#RandomForestClassifier, utiliza n_estimators que es
#el número de árboles que se generarán para el datset de entrenamiento.
classifier = RandomForestClassifier(n_estimators=200)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

#### Predicción y evaluación de modelos Random Forest

In [11]:
rfc_pred = classifier.predict(X_test)
print(confusion_matrix(y_test,rfc_pred))
print(classification_report(y_test,rfc_pred))

[[29 10]
 [ 2 50]]
              precision    recall  f1-score   support

           0       0.94      0.74      0.83        39
           1       0.83      0.96      0.89        52

    accuracy                           0.87        91
   macro avg       0.88      0.85      0.86        91
weighted avg       0.88      0.87      0.87        91



Este modelo aleatorio logró una tasa de exactitud del 86%, que es un poco superior que lo encontrado en el modelo de árbol de decisión. 

### Vamos a predecir el pronóstico de la diabetes usando Random Forest

In [12]:
dataset = pd.read_csv('../Datos/diabetes.csv')
dataset.head()

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,0
1,1147438,8,92,93,47,36,21.240576,0.158365,23,0
2,1640031,7,115,47,52,35,41.511523,0.079019,23,0
3,1883350,9,103,78,25,304,29.582192,1.282870,43,1
4,1424119,1,85,59,27,35,42.604536,0.549542,22,0


In [13]:
dataset['Diabetic'].value_counts()

0    10000
1     5000
Name: Diabetic, dtype: int64

Este dataset tiene nueve características que describen los valores de las observaciones, y la ultima columna Diabetic, representa la etiqueta si una persona es diabética "1" y si no es diabética es "0"

In [14]:
#verificamos si se tienen valores nulos
dataset.isnull().sum()

PatientID                 0
Pregnancies               0
PlasmaGlucose             0
DiastolicBloodPressure    0
TricepsThickness          0
SerumInsulin              0
BMI                       0
DiabetesPedigree          0
Age                       0
Diabetic                  0
dtype: int64

#### Dividimos el conjunto de entrenamiento y el de prueba

In [15]:
X = dataset.iloc[:,1:-1]
y = dataset.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,
random_state = 0)

#### Clasificador de entrenamiento

In [16]:
from sklearn.ensemble import RandomForestClassifier
#Utilizaremos 10 árboles de desición para comenzar el clasificador
classifier = RandomForestClassifier(n_estimators=10)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10)

In [17]:
rfc_pred = classifier.predict(X_test)
print(classification_report(y_test,rfc_pred))


              precision    recall  f1-score   support

           0       0.93      0.96      0.94      2500
           1       0.91      0.86      0.88      1250

    accuracy                           0.92      3750
   macro avg       0.92      0.91      0.91      3750
weighted avg       0.92      0.92      0.92      3750



El clasificador de bosques aleatorio se desempeño significativamente bien con este dataset, logrando una tase de exactitud del 93%. 

#### Validación cruzada (Cross-Validation)

En los anteriores modelos dividimos nuestro conjunto de datos aleatorios en dos conjuntos: Entrenamiento y de prueba. Sin embargo, existe una limitación en esta técnica. Si sacrificamos alrededor del 20% al 25% de nuestros datos para evaluar nuestro sistema, es posible que nuestro modelo no pueda entrenarse adecuadamente para los desafíos del mundo real. Cierta información valiosa podría perderse cuando dividimos el conjunto de datos. Esto dará como resultado un mayor sesgo en nuestro sistema. Este problema se agrava cuando trabajamos con un conjunto de datos pequeños.

Para superar esto usamos la técnica Cross-validation, para hacer esto, dividiremos el dataset en varios subconjuntos, luego entrenamos y evaluamos nuestros modelo con varias combinaciones de estos subconjuntos.


In [18]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=10)
cross_validator = cross_validate(classifier, X, y, cv=5,
    scoring = 'accuracy',
    return_estimator = 'true')
print(cross_validator['test_score'])

[0.93066667 0.93033333 0.93366667 0.927      0.93266667]


La clase cross_validate requiere el clasificador, características, etiquetas y cv que es el número de K-folds. 
return_estimator nos permite recuperar el mejor estimador.

Podemos observar en el resultado que se han entrenado cinco modelos con combinaciones de datos de prueba y entrenamiento. El ultimo tiene el mayor "test_score", podemos utilizar "best_model = cross_validator[‘estimator’][-1]" para recuperar el mejor modelo. "best_model es el clasificador de bosques aleatorios que se puede usar para futuras predicciones.

In [19]:
best_model = cross_validator['estimator'][-1]